In [1]:
# importar librerias
import pandas as pd

### ETL Migración mujeres

In [8]:
# migrantes mujeres

# Función para omitir filas en el archivo CSV
def skip_row(row, paises_a_filtrar):
    return any(pais in row for pais in paises_a_filtrar)

# Función para procesar archivos CSV de destino y origen
def procesar_archivos_csv(archivo_destino, archivo_origen, archivo_salida, df_codigos_pais, anios_a_filtrar):
    # Leer archivos de destino y origen
    df_destino = pd.read_csv(archivo_destino)
    df_origen = pd.read_csv(archivo_origen)
    
    # Renombrar columnas
    df_destino.rename(columns={'Region, development group, country or area': 'País', 'Year': 'Año', 'Location code': 'Código de ubicación', 'Numero de Migrantes': 'Mujeres_destino'}, inplace=True)
    df_origen.rename(columns={'Region': 'País', 'Year': 'Año', 'Location code': 'Código de ubicación', 'Numero de Migrantes': 'Mujeres_origen'}, inplace=True)
    
    # Filtrar por años
    df_destino = df_destino[~df_destino['Año'].isin(anios_a_filtrar)]
    df_origen = df_origen[~df_origen['Año'].isin(anios_a_filtrar)]
    
    # Combinar los DataFrames de destino y origen
    df_combined = df_destino.merge(df_origen, on=['País', 'Código de ubicación', 'Año'], how='outer')
    df_combined['País'] = df_combined['País'].str.replace('*', '').str.strip()
    
    # Realizar la fusión con los códigos de país
    df_combined = df_combined.merge(df_codigos_pais, left_on='País', right_on='Pais', how='left')
    df_combined.drop(columns=['Código de ubicación', 'Pais'], inplace=True)
    
    # Reorganizar columnas
    col_orden = ['Año', 'País', 'Id_pais', 'Mujeres_destino', 'Mujeres_origen']
    df_combined = df_combined[col_orden]
    
    # cambiar valores NaN con 0
    df_combined = df_combined.fillna(0)
    
    # Guardar el DataFrame limpio en un archivo CSV
    df_combined.to_csv(archivo_salida, index=False)

# Uso de la función para procesar los archivos y crear el archivo final

columnas_a_eliminar = ['Notes', 'Type of data']
anios_a_filtrar_destino = [1990, 1995]
anios_a_filtrar_origen = [1990, 1995]
archivo_destino = 'destino_mujeres_pais.csv'
archivo_origen = 'origen_mujeres_clean.csv'
archivo_salida = 'migrantes_mujeres250.csv'
paises_a_filtrar = ['WORLD', 'Sub-Saharan Africa', 'Northern Africa and Western Asia',
                         'Central and Southern Asia', 'Eastern and South-Eastern Asia',
                         'Latin America and the Caribbean',
                         'Oceania (excluding Australia and New Zealand)',
                         'Australia and New Zealand', 'Europe and Northern America',
                         'Developed regions', 'Less developed regions',
                         'Less developed regions, excluding least developed countries',
                         'Less developed regions, excluding China',
                         'Least developed countries',
                         'Land-locked Developing Countries (LLDC)',
                         'Small island developing States (SIDS)',
                         'High-income countries', 'Middle-income countries',
                         'Upper-middle-income countries', 'Lower-middle-income countries',
                         'Low-income countries', 'AFRICA', 'Eastern Africa',
                         'Northern Africa', 'Middle Africa', 'Southern Africa',
                         'Western Africa', 'ASIA','Central Asia', 'Eastern Asia', 
                         'South-Eastern Asia', 'Southern Asia', 'Western Asia', 'EUROPE',
                         'Eastern Europe', 'Northern Europe', 'Southern Europe', 'Western Europe',
                         'Caribbean', 'Central America','South America', 'NORTHERN AMERICA', 'OCEANIA',
                         'Melanesia', 'Micronesia', 'Polynesia*', 'OTHER']

df_codigos_pais = pd.read_csv('codigos_pais.csv')  

procesar_archivos_csv(archivo_destino, archivo_origen, archivo_salida, df_codigos_pais, anios_a_filtrar_destino)

### ETL Migración origen y destino

In [ ]:
# Combinado origen y destino

# Funcion para procesar archivo migracion
def procesar_archivos_migracion(archivo_origen, archivo_destino, archivo_salida):

    # Lee los archivos CSV de origen y destino
    df1 = pd.read_csv(archivo_origen)
    df2 = pd.read_csv(archivo_destino)

    # Combinación de DataFrames
    df_combined = pd.merge(df1, df2, on=['País', 'Código de País', 'Año'], suffixes=('_origen', '_destino'))

    # Renombrar columnas
    df_combined.rename(columns={
        'País': 'Pais',
        'Código de País': 'Id_Pais',
        'Hombres origen': 'Hombres_origen',
        'Mujeres origen': 'Mujeres_origen',
        'Hombres destino': 'Hombres_destino',
        'Mujeres destino': 'Mujeres_destino'
    }, inplace=True)

    # Eliminar columnas de código de ubicación
    df_combined.drop(columns=['Código de ubicación_origen', 'Código de ubicación_destino'], inplace=True)

    # Reordenar columnas
    col_orden = ['Año', 'Pais', 'Id_Pais', 'Hombres_origen', 'Mujeres_origen', 'Hombres_destino', 'Mujeres_destino']
    df_combined = df_combined[col_orden]

    # Convertir columnas numéricas a tipo float
    columnas_numericas = ['Año', 'Hombres_origen', 'Mujeres_origen', 'Hombres_destino', 'Mujeres_destino']
    df_combined[columnas_numericas] = df_combined[columnas_numericas].astype(int)

    # Limpiar valores en la columna 'Pais'
    df_combined['Pais'] = df_combined['Pais'].str.replace('*', '').str.strip()

    # Guardar el DataFrame en un nuevo archivo CSV
    df_combined.to_csv(archivo_salida, index=False)

# Uso de la función
procesar_archivos_migracion('origen_H_M.csv', 'destino_H_M.csv', 'migracion_origen_destino.csv')

### ETL Población

In [ ]:
# poblacion

def procesar_archivo_poblacion(archivo_entrada, archivo_salida):
    # Lee el archivo CSV de entrada
    df = pd.read_csv(archivo_entrada)

    # Elimina filas específicas
    eliminar_filas = [4557, 4558, 4559, 4560, 4561]
    df = df.drop(eliminar_filas)

    # Reinicia el índice
    df.reset_index(drop=True, inplace=True)

    # Elimina la columna 'Time Code'
    df = df.drop(columns=['Time Code'])

    # Renombra columnas
    df = df.rename(columns={
        'Time': 'Año',
        'Country Name': 'Pais',
        'Country Code': 'Id_Pais',
        'Population, total [SP.POP.TOTL]': 'Poblacion_total',
        'Population, male [SP.POP.TOTL.MA.IN]': 'Poblacion_masculina',
        'Population, female [SP.POP.TOTL.FE.IN]': 'Poblacion_femenina',
    })

    # Formato de visualización de números
    pd.options.display.float_format = '{:.2f}'.format

    # Guarda el DataFrame en un nuevo archivo CSV
    df.to_csv(archivo_salida, index=False)

# Uso de la función
procesar_archivo_poblacion('poblacion_HM.csv', 'poblacion_H_M.csv')

### ETL Hombres origen y destino

In [ ]:
# origen y Destino Hombres

def procesar_archivo(archivo_entrada, archivo_salida, genero, anios_a_filtrar, paises_a_filtrar):
    # Lee el archivo CSV de entrada
    df = pd.read_csv(archivo_entrada)

    # Filtra por años
    df = df[~df['Year'].isin(anios_a_filtrar)]

    # Filtra por género
    df_genero = df[df['Género'] == genero]

    if 'Número de Migrantes' in df_genero.columns:
        # Renombra la columna 'Número de Migrantes' a 'Migrantes Genero'
        df_genero = df_genero.rename(columns={'Número de Migrantes': f'Migrantes {genero}'})
    else:
        # Renombra la columna 'Numero de Migrantes' a 'Mujeres' o 'Hombres' según el género
        df_genero = df_genero.rename(columns={'Numero de Migrantes': genero})

    # Elimina la columna 'Género'
    df_genero.drop('Género', axis=1, inplace=True)

    # Filtra los países a excluir
    paises_a_filtrar = [pais.strip().upper() for pais in paises_a_filtrar]
    df_genero = df_genero[~df_genero['Region'].str.strip().str.upper().isin(paises_a_filtrar)].copy()

    # Reinicia el índice
    df_genero.reset_index(drop=True, inplace=True)

    # Renombrar columnas
    df_genero.rename(columns={'Region': 'Pais', 'Year': 'Año', 'Location code': 'Código de ubicación'}, inplace=True)

    # Verifica si las columnas existen antes de intentar eliminarlas
    if 'Notes' in df_genero.columns and 'Type of data' in df_genero.columns:
        # Elimina las columnas 'Notes' y 'Type of data'
        df_genero.drop(['Notes', 'Type of data'], axis=1, inplace=True)

    if 'Año' in df_genero.columns:
        # Convierte la columna 'Año' a tipo entero
        df_genero['Año'] = df_genero['Año'].astype(int)

    # Guarda el DataFrame limpio en un nuevo archivo CSV
    df_genero.to_csv(archivo_salida, index=False)

paises_a_filtrar = ['WORLD', 'Sub-Saharan Africa', 'Northern Africa and Western Asia',
                     'Central and Southern Asia', 'Eastern and South-Eastern Asia',
                     'Latin America and the Caribbean',
                     'Oceania (excluding Australia and New Zealand)',
                     'Australia and New Zealand', 'Europe and Northern America',
                     'Developed regions', 'Less developed regions',
                     'Less developed regions, excluding least developed countries',
                     'Less developed regions, excluding China',
                     'Least developed countries',
                     'Land-locked Developing Countries (LLDC)',
                     'Small island developing States (SIDS)',
                     'High-income countries', 'Middle-income countries',
                     'Upper-middle-income countries', 'Lower-middle-income countries',
                     'Low-income countries', 'AFRICA', 'Eastern Africa',
                     'Northern Africa', 'Middle Africa', 'Southern Africa',
                     'Western Africa', 'ASIA','Central Asia', 'Eastern Asia', 
                     'South-Eastern Asia', 'Southern Asia', 'Western Asia', 'EUROPE',
                     'Eastern Europe', 'Northern Europe', 'Southern Europe', 'Western Europe',
                     'Caribbean', 'Central America','South America', 'NORTHERN AMERICA', 'OCEANIA',
                     'Melanesia', 'Micronesia', 'Polynesia*', 'OTHER']    

# Uso de la función para procesar archivos de origen
procesar_archivo('origen.csv', 'origen_hombres.csv', 'Hombres', [1990, 1995], paises_a_filtrar)

# Uso de la función para procesar archivos de destino
procesar_archivo('destino.csv', 'destino_hombres.csv', 'Hombres', [1990.1, 1995.1, 1990.2, 1995.2], paises_a_filtrar)

### ETL Indicadores

In [ ]:
# indicadores

def procesar_datos_indicadores(archivo_entrada, archivo_salida):
    # Leer el archivo CSV de entrada
    df = pd.read_csv(archivo_entrada)

    # Eliminar columnas innecesarias
    columnas_a_excluir = ['Time Code']
    df = df.drop(columns=columnas_a_excluir)

    # Renombrar columnas
    df = df.rename(columns={
        'Time': 'Año',
        'Country Name': 'Pais',
        'Country Code': 'Codigo_Pais',
        'GDP growth (annual %) [NY.GDP.MKTP.KD.ZG]': 'Crecimiento_PIB',
        'Unemployment, total (% of total labor force) (modeled ILO estimate) [SL.UEM.TOTL.ZS]': 'Tasa_desempleo',
        'Inflation, GDP deflator (annual %) [NY.GDP.DEFL.KD.ZG]': 'Inflacion_PIB',
        'Intentional homicides (per 100,000 people) [VC.IHR.PSRC.P5]': 'Homicidios_intencionales',
        'Battle-related deaths (number of people) [VC.BTL.DETH]': 'Muertes_Conflicto',
        'Control of Corruption: Percentile Rank [CC.PER.RNK]': 'Control_Corrupcion',
        'CO2 emissions (kt) [EN.ATM.CO2E.KT]': 'Emisiones_CO2',
        'Life expectancy at birth, total (years) [SP.DYN.LE00.IN]': 'Esperanza_vida',
        'Net migration [SM.POP.NETM]': 'Migracion_neta',
        'Population, total [SP.POP.TOTL]': 'Poblacion_total'
    })

    # Reemplazar '..' con 0
    df.replace('..', 0, inplace=True)

    # Eliminar filas específicas
    eliminar_filas = [4557, 4558, 4559, 4560, 4561]
    df = df.drop(eliminar_filas)

    # Reiniciar el índice
    df.reset_index(drop=True, inplace=True)

    # Convertir columnas numéricas de texto a enteros
    columnas_a_convertir = [col for col in df.columns if col not in ['Año', 'Pais', 'Codigo_Pais']]
    for columna in columnas_a_convertir:
        if df[columna].dtype == 'O':
            df[columna] = df[columna].str.replace(',', '').str.replace('%', '').astype(int)

    # Rellenar valores nulos con 0
    df.fillna(0, inplace=True)

    # Guardar el DataFrame limpio en un nuevo archivo CSV
    df.to_csv(archivo_salida, index=False)

# Uso de la función para procesar datos de indicadores
procesar_datos_indicadores('indicadores_migracion.csv', 'indicadores.csv')

### ETL Refugiados origen y destino

In [ ]:
# Refugiados pais origen y destino

def procesar_datos_refugiados(archivo_entrada, archivo_salida, tipo_dato):
    # Lee el archivo CSV de entrada
    df = pd.read_csv(archivo_entrada)

    # Renombra columnas comunes
    df.rename(columns={'Entity': 'Pais', 'Code': 'Id_Pais', 'Year': 'Año'}, inplace=True)

    if tipo_dato == 'asilo':
        # Renombra la columna específica para datos de asilo
        df.rename(columns={'Refugee population by country or territory of asylum': 'Poblacion_pais_asilo'}, inplace=True)
    elif tipo_dato == 'origen':
        # Renombra la columna para datos de origen
        df.rename(columns={'Refugee population by country or territory of origin': 'Poblacion_pais_origen'}, inplace=True)

    # Selecciona las columnas de interés
    col_orden = ['Año', 'Pais', 'Id_Pais']
    if tipo_dato == 'asilo':
        col_orden.append('Poblacion_pais_asilo')
    elif tipo_dato == 'origen':
        col_orden.append('Poblacion_pais_origen')
    
    df = df[col_orden]

    # Filtra por años
    años_a_mantener = range(2000, 2021)
    df = df.loc[df['Año'].isin(años_a_mantener)]

    # Paises a filtrar
    paises_a_filtrar = ['East Asia and Pacific (WB)', 'Europe and Central Asia (WB)', 'European Union (27)',
                        'High-income countries', 'Latin America and Caribbean (WB)', 'Low-income countries',
                        'Lower-middle-income countries', 'Middle East and North Africa (WB)', 'Middle-income countries',
                        'North America (WB)', 'Upper-middle-income countries', 'South Asia (WB)',
                        'Sub-Saharan Africa (WB)', 'Upper-middle-income countries','West Bank and Gaza', 'World']

    paises_a_filtrar = [pais.strip().upper() for pais in paises_a_filtrar]

    # Filtra los países a excluir
    df = df[~df['Pais'].str.strip().str.upper().isin(paises_a_filtrar)].copy()

    # Reinicia el índice
    df.reset_index(drop=True, inplace=True)

    # Guarda el DataFrame limpio en un nuevo archivo CSV
    df.to_csv(archivo_salida, index=False)

# Uso de la función para procesar datos de asilo
procesar_datos_refugiados('refugiados_paisasilo.csv', 'refugiados_pais_asilo250.csv', 'asilo')

# Uso de la función para procesar datos de origen
procesar_datos_refugiados('refugiados_paisorigen.csv', 'refugiados_pais_origen250.csv', 'origen')